<a href="https://colab.research.google.com/github/locastre/pyCERR/blob/main/autosegment_CT_Heart_OARs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The pyCERR DeepLab Heart Substurcture CT OAR Segmentation Model

In this tutorial, we will demonstrate how to apply a pre-trained AI model to segment the Heart sub-structures on a lung CT scan using pyCERR.

## Software Requirements
* Python>=3.8
* Applying this model requires access to a GPU.  

## Input Data Requirements
* RT planning DICOM CT
* RTSTRUCT with lung contour

## AI model
* The segmentation model was trained and validated on CT scans used for RT planning. It does not work optimally on diagnostic CTs or scans in positions other than Head First Supine.
* The trained model is distributed along with python libraries and other dependencies via a conda package.

### Running the model

Update locations of input data and model directorues in section 2 of this notebook.
* Conda packge is location: condaEnvDir
* Inference script location: wrapperPath
* Inference script args
```python
!python {wrapperPath} {input_nii_directory} {output_nii_directory}
```

### License
By downloading the software you are agreeing to the following terms and conditions as well as to the Terms of Use of CERR software.

THE SOFTWARE IS PROVIDED "AS IS" AND CERR DEVELOPMENT TEAM AND ITS COLLABORATORS DO NOT MAKE ANY WARRANTY, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE, NOR DO THEY ASSUME ANY LIABILITY OR RESPONSIBILITY FOR THE USE OF THIS SOFTWARE.
    
This software is for research purposes only and has not been approved for clinical use.

Software has not been reviewed or approved by the Food and Drug Administration, and is for non-clinical, IRB-approved Research Use Only. In no event shall data or images generated through the use of the Software be used in the provision of patient care.

You may publish papers and books using results produced using software provided that you reference the appropriate citations:
*  Heart sub-structures model: https://doi.org/10.1016/j.phro.2020.05.009
*  CERR library of model implementations: https://doi.org/10.1016/j.ejmp.2020.04.011
*  CERR software: https://doi.org/10.1118/1.1568978
*  CERR radiomics: https://doi.org/10.1002/mp.13046


YOU MAY NOT DISTRIBUTE COPIES of this software, or copies of software derived from this software, to others outside your organization without specific prior written permission from the CERR development team except where noted for specific software products.

All Technology and technical data delivered under this Agreement are subject to US export control laws and may be subject to export or import regulations in other countries. You agree to comply strictly with all such laws and regulations and acknowledge that you have the responsibility to obtain such licenses to export, re-export, or import as may be required after delivery to you.



##  Define paths for input DICOM, output segmentation and session directories

Specify paths to the DICOM input data, desired output directory, and temporary (session) directory used to store intermediate results. The input data is structures such that DICOM per patient scan is in an individual directory.

&nbsp;&nbsp;Input Directory  
&nbsp;&nbsp;&nbsp;&nbsp;Pat1  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;img1.dcm  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;img2.dcm  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;....  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;....  
&nbsp;&nbsp;&nbsp;&nbsp;Pat2  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;img1.dcm  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;img2.dcm  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;....  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;....  

In [40]:
import os
# work dir
#workDir = r'/home/jupyter' #
workDir = r'/content' #for Colab # r'AI_workshop/heart_model' # local

# Location of dicom data
#dataDownloadDir = os.path.join(workDir,'sampleData')

#os.makedirs(dataDownloadDir,exist_ok=True)

## Install segmentation wrapper, pyCERR and other dependencies

### Installaion of segmentation wrapper, Python environment and network weights handled by CERR-developed `model_installer`

In [41]:
os.chdir(workDir)
!git clone https://github.com/cerr/model_installer.git
os.chdir(os.path.join(workDir,'model_installer'))
!./installer.sh -h

Cloning into 'model_installer'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 49 (delta 15), reused 26 (delta 5), pack-reused 0
Receiving objects: 100% (49/49), 14.14 KiB | 2.83 MiB/s, done.
Resolving deltas: 100% (15/15), done.
      ___           ___           ___           ___     
     /  /\         /  /\         /  /\         /  /\    
    /  /:/        /  /:/_       /  /::\       /  /::\   
   /  /:/        /  /:/ /\     /  /:/\:\     /  /:/\:\  
  /  /:/  ___   /  /:/ /:/_   /  /:/~/:/    /  /:/~/:/  
 /__/:/  /  /\ /__/:/ /:/ /\ /__/:/ /:/___ /__/:/ /:/___
 \  \:\ /  /:/ \  \:\/:/ /:/ \  \:\/:::::/ \  \:\/:::::/
  \  \:\  /:/   \  \::/ /:/   \  \::/~~~~   \  \::/~~~~ 
   \  \:\/:/     \  \:\/:/     \  \:\        \  \:\     
    \  \::/       \  \::/       \  \:\        \  \:\    
     \__\/         \__\/         \__\/         \__\/    
 
Medical Physics Department, Memorial S

In [42]:
%%capture

modelOpt = '1' #CT_Lung_SMIT model
pythonOpt = 'C' #download and use pre-packaged Conda environment

! source ./installer.sh -m {modelOpt} -d {workDir} -p {pythonOpt}

      ___           ___           ___           ___     
     /  /\         /  /\         /  /\         /  /\    
    /  /:/        /  /:/_       /  /::\       /  /::\   
   /  /:/        /  /:/ /\     /  /:/\:\     /  /:/\:\  
  /  /:/  ___   /  /:/ /:/_   /  /:/~/:/    /  /:/~/:/  
 /__/:/  /  /\ /__/:/ /:/ /\ /__/:/ /:/___ /__/:/ /:/___
 \  \:\ /  /:/ \  \:\/:/ /:/ \  \:\/:::::/ \  \:\/:::::/
  \  \:\  /:/   \  \::/ /:/   \  \::/~~~~   \  \::/~~~~ 
   \  \:\/:/     \  \:\/:/     \  \:\        \  \:\     
    \  \::/       \  \::/       \  \:\        \  \:\    
     \__\/         \__\/         \__\/         \__\/    
 
Medical Physics Department, Memorial Sloan Kettering Cancer Center, New York, NY
 
Welcome to the CERR segmentation model installer! For usage information, run with -h flag
 
git clone https://github.com/cerr/CT_cardiac_structures_deeplab.git
Cloning into 'CT_cardiac_structures_deeplab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109)

In [43]:
%%capture
! pip install pyxnat
! pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git@testing"

In [44]:
# Segmentation wrapper paths
wrapperInstallDir = os.path.join(workDir, 'CT_cardiac_structures_deeplab')
wrapperPath = os.path.join(wrapperInstallDir, 'model_wrapper','runSegmentation.py')

# Location of Heart OAR conda environment
condaEnvDir = os.path.join(workDir, 'CT_cardiac_structures_deeplab','conda-pack')

# Path to conda environment activate script
condaActivateScript = os.path.join(condaEnvDir, 'bin', 'activate')

#location of input DICOM folders
inputDicomPath = os.path.join(workDir,'input')
if not os.path.exists(inputDicomPath):
  os.mkdir(inputDicomPath)

#location of output RTSTRUCT file
outputDicomPath = os.path.join(workDir, 'output')
if not os.path.exists(outputDicomPath):
  os.mkdir(outputDicomPath)

#temp session directory
sessionPath = os.path.join(workDir, 'session')
if not os.path.exists(sessionPath):
  os.mkdir(sessionPath)


# Download planning CT DICOM data for processing

### Option 1: Download data from XNAT source (`getXNATData`)

In [45]:
from pyxnat import Interface
import urllib3
import shutil
urllib3.disable_warnings()

from pyxnat import Interface
def getXNATData(xhost,user,scandict,downloadDir):
  xnat = Interface(xhost, user, verify=False)
  os.makedirs(downloadDir, exist_ok=True)
  expdirlist = []
  for scan_entry in scandict:
    proj = scan_entry['proj']
    subj = scan_entry['subj']
    exp = scan_entry['exp']
    scan_list = scan_entry['scan_list']
    expdir = os.path.join(downloadDir,exp)
    expdirlist.append(expdir)
    os.makedirs(expdir, exist_ok = True)
    xexp = xnat.select.project(proj).subject(subj).experiment(exp)
    for scan in scan_list:
      try:
        xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('DICOM').get(downloadDir,extract=True)
      except:
        xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('secondary').get(downloadDir,extract=True)
    for dcmfolder in ['DICOM','secondary']:
      dcmlist = glob(os.path.join(downloadDir,dcmfolder,'*.dcm'))
      print(dcmlist)
      for dcm in dcmlist:
        shutil.move(dcm, expdir)
  for dcmfolder in ['DICOM','secondary']:
    if os.path.exists(os.path.join(downloadDir,dcmfolder)):
      os.rmdir(os.path.join(downloadDir,dcmfolder))
    if os.path.exists(os.path.join(downloadDir,dcmfolder + '.zip')):
      os.remove(os.path.join(downloadDir,dcmfolder + '.zip'))
  xnat.disconnect()
  return expdirlist

In [46]:
xhost = 'https://pixnat.mskcc.org'
user = 'locastre'
scandict = [{'proj':'RTOG0617','subj':'SPI_XNAT_S00196','exp':'SPI_XNAT_E00348', 'scan_list':['1','2']},{'proj':'RTOG0617','subj':'SPI_XNAT_S00195','exp':'SPI_XNAT_E00347','scan_list':['1','2']}]

dcmdirlist = getXNATData(xhost,user,scandict,inputDicomPath)

··········
['/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-92-1mwrbch.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-51-1qoylpv.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-32-1cemjsq.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-37-12pneeu.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-90-1sfmeq5.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-62-xi286m.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-64-88rrvo.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-41-qvlxuv.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-93-1878ffd.dcm', '/content/input/DICOM/1.3.6.1.4.1.145

### Option 2: Download data from other HTTP source to `inputDicomDir`

In [36]:
dataUrl='http://path.to/data'

dataDownloadDir = os.path.join(workDir,'tmp')
os.makedirs(dataDownloadDir, exist_ok=True)
os.chdir(dataDownloadDir)

! wget -O sampleData.gz -L {dataUrl}
! tar xf sampleData.gz -C {inputDicomDir}
! rm sampleData.gz

#unpack and move

--2024-06-17 19:04:29--  http://path.to/data
Resolving path.to (path.to)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘path.to’
tar: This does not look like a tar archive

gzip: stdin: unexpected end of file
tar: Child returned status 1
tar: Error is not recoverable: exiting now


# Function Definitions: Data pre- and post-processing using pyCERR

## `processInputData`: Crop scan to Lung extents

In [47]:
from cerr.dataclasses import structure as cerrStr
from cerr.contour import rasterseg as rs
from cerr.utils import aiPipeline, mask
import numpy as np

def processInputData(scanNum, planC, lungNameList=['LUNG_TOTAL', 'LUNG_L', 'LUNG_R']):

    if isinstance(lungNameList, str):
        lungNameList = [lungNameList]

    # Extract scanArray
    scan3M = planC.scan[scanNum].getScanArray()
    mask3M = np.zeros(scan3M.shape, dtype=bool)

    # List of Structure names
    strNames = [s.structureName for s in planC.structure]
    numOrigStructs = len(strNames)

    # Get total lung mask
    for lungName in lungNameList:
        lungInd = cerrStr.getMatchingIndex(lungName.upper(), strNames, 'exact')
        if len(lungInd) > 0:
            # Get lung extents
            mask3M = mask3M | rs.getStrMask(lungInd[0], planC)

    if not np.any(mask3M):
        raise Exception('Lung contour name did not match any structures in planC')

    # Create cropped scan
    rmin,rmax,cmin,cmax,smin,smax,_ = mask.compute_boundingbox(mask3M)
    x,y,z = planC.scan[0].getScanXYZVals()
    xCropV = x[cmin:cmax]
    yCropV = y[rmin:rmax]
    zCropV = z[smin:smax]
    scan3M = planC.scan[0].getScanArray()
    scanCrop3M = scan3M[rmin:rmax,cmin:cmax,smin:smax]

    return scanCrop3M, (xCropV, yCropV, zCropV)


## `postProcAndImportSeg`: Import AI segmentations to planC and retain only the largest connected component for each structure

In [48]:
#Import label map to CERR
from glob import glob
from cerr import plan_container as pc

atriaLabelDict = {1: 'DL_Atria'}
heartSubSegDict = {2: 'DL_AORTA', 3: 'DL_LA',
                   4: 'DL_LV', 5: 'DL_RA',
                   6: 'DL_RV', 7: 'DL_IVC',
                   8: 'DL_SVC', 9: 'DL_PA'}
heartSegDict = {1: 'DL_heart'}
periLabelDict = {1: 'DL_Pericardium'}
ventriLabelDict = {1: 'DL_Ventricles'}

def postProcAndImportSeg(outputDir,procScanNum,scanNum,planC):
    niiGlob = glob(os.path.join(outputDir,'*.nii.gz'))
    for segFile in niiGlob:
        print('Importing ' + segFile + '...')
        # Get segFile name
        if 'heart.nii.gz' in segFile:
            strToLabelMap = heartSubSegDict
        elif 'heartStructure.nii.gz' in segFile:
            strToLabelMap = heartSegDict
        elif 'atria.nii.gz' in segFile:
            strToLabelMap = atriaLabelDict
        elif 'pericardium.nii.gz' in segFile:
            strToLabelMap = periLabelDict
        elif 'ventricles.nii.gz' in segFile:
            strToLabelMap = ventriLabelDict
        numLabel = len(strToLabelMap)
        numStrOrig = len(planC.structure)
        planC = pc.load_nii_structure(segFile, scanNum, planC, \
                                  labels_dict = strToLabelMap)
        numStructs = len(planC.structure)
        cpyStrNumV = np.arange(numStrOrig,numStructs)
        numConnComponents = 1
        for structNum in cpyStrNumV:
            _, planC = cerrStr.getLargestConnComps(structNum, numConnComponents, planC, \
                                            saveFlag=True, replaceFlag=True)

    return planC

# Run Segmentation: Generate OARs for all the CT scan folders located at `inputDicomPath`

In [51]:
#%%capture
import subprocess
import numpy as np
import cerr
from cerr import plan_container as pc
from cerr.dataclasses import scan as cerrScn
from cerr.utils.aiPipeline import createSessionDir
from cerr.dcm_export import rtstruct_iod

structFileName = 'RTSTRUCT.dcm'

# Loop over DICOM data folders
#fileList = os.listdir(inputDicomPath)
#numFiles = len(fileList)
folderList = glob(os.path.join(inputDicomPath,'*'))
modality = 'CT SCAN'
lungNameList = ['LUNG_TOTAL', 'LUNG_CNTR', 'LUNG_IPSI']

#for iFile in range(numFiles):
for dcmDir in folderList:

    fname = os.path.basename(dcmDir)
    # Create session dir to store temporary data
    modInputPath, modOutputPath = createSessionDir(sessionPath, dcmDir)

    # Import DICOM to planC
    planC = pc.load_dcm_dir(dcmDir)

    # Identify scan index in  planC
    scanIdS = {"imageType": modality}
    matchScanV = aiPipeline.getScanNumFromIdentifier(scanIdS, planC, False)
    scanNum = matchScanV[0]

    # Pre-process data
    procScan3M, resizeGridS = processInputData(scanNum, planC, lungNameList)
    planC = pc.import_scan_array(procScan3M, resizeGridS[0], \
            resizeGridS[1], resizeGridS[2], modality, scanNum, planC)
    procScanNum = len(planC.scan) - 1

    # Export inputs to NIfTI
    scanFilename = os.path.join(modInputPath, f"{fname}_scan_3D.nii.gz")
    planC.scan[procScanNum].save_nii(scanFilename)

    numOrigStructs = len(planC.structure)

    # Apply model
    runScript = "source " + condaActivateScript + " && python " + wrapperPath \
                  + " " + modInputPath + " " + modOutputPath
    print(runScript)
    subprocess.run(runScript,
                 capture_output=False,
                  shell=True,
                  executable="/bin/bash")

    # Import results to planC
    planC = postProcAndImportSeg(modOutputPath,procScanNum,scanNum,planC)

    numStructs = len(planC.structure)

    # Export segmentations to DICOM
    structFileName = fname + '_AI_seg_RTSTRUCT.dcm'
    structFilePath = os.path.join(outputDicomPath,structFileName)
    structNumV = np.arange(numOrigStructs, numStructs)
    indOrigV = np.array([cerrScn.getScanNumFromUID(planC.structure[structNum].assocScanUID, planC) for structNum in structNumV], dtype=int)
    origIndsToExportV = structNumV[indOrigV == scanNum]
    seriesDescription = "pyCERR CT_Heart_OAR AI-Generated"
    exportOpts = {'seriesDescription': seriesDescription}
    rtstruct_iod.create(origIndsToExportV,structFilePath,planC,exportOpts)

('0617-259694', '0617-259694', '1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111', '1.3.6.1.4.1.14519.5.2.1.6329.6468.208597257645079747725955792395', 'CT', 'CT', 'CT', 'CT', 'CT', 'CT', 'CT')
('0617-259694', '0617-259694', '1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111', '1.3.6.1.4.1.14519.5.2.1.6329.6468.370049441999407001621950329800', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT')
source /content/CT_cardiac_structures_deeplab/conda-pack/bin/activate && python /content/CT_cardiac_structures_deeplab/model_wrapper/runSegmentation.py /content/session/sessionSPI_XNAT_E0034820929833.447/inputNii /content/session/sessionSPI_XNAT_E0034820929833.447/outputNii
Importing /content/session/sessionSPI_XNAT_E0034820929833.447/outputNii/SPI_XNAT_E00348_scan_3D_ventricles.nii.gz...
Importing /content/session/sessionSPI_XNAT_E0034820929833.447/outputNii/SPI_XNAT_E00348_scan_3D_heart.nii.gz...
Importing /content/session/sessionSP

# Optional - Offload the `outputDicomPath` to GCP Workspace bucket

In [ ]:
# workspaceBucket = os.environ['WORKSPACE_BUCKET']
# !gcloud storage cp -r {outputDicomPath} {workspaceBucket}

# Visualize results for the last CT scan using Napari Viewer

## Display using `matplotlib`

In [29]:
from cerr.viewer import showMplNb

showMplNb(scanNum, origIndsToExportV, planC,\
          windowCenter=-400, windowWidth=2000)

interactive(children=(IntSlider(value=48, description='slcNum', max=144, min=-48), Text(value='axial', descrip…

interactive(children=(IntSlider(value=256, description='slcNum', max=768, min=-256), Text(value='sagittal', de…

interactive(children=(IntSlider(value=256, description='slcNum', max=768, min=-256), Text(value='coronal', des…